# BUSINESS CONTEXT

**Company Name:** DHAUZ Real State

**Product/Service:** The company offers a service that estimates apartments price through their features and characteristics, serving as a guide to customers wanting to sell their apartments.

**Business Model:**

- The service price is based on two components:

    - 1 - A minimum price per estimate, which is R$ 200,00
    - 2 - Based on the error of the estimate, which is based on the MAPE. If the error is less than 0.05, it is added R$ 400,00. For every 1% more error, it is subtracted R$ 20,00. If the error is greater than 20%, than the customer only pays the minimun price

**Current Situation:**

- The estimate is done by real state agent
- The current MAPE is around 18%, varying from 30% and 8%

**Problem:**

- The company started this business model to validade their service, and now is trying to improve their profit, delivering a more accurate estimate.

**Proposed Solution:**

- Create a Linear Model to estimate apartment price, to achieve a more accurate estimate and helping real state agents do their job.

**Main Questions to be answered:**

- How much is the average error?
- How much more profit DHAUZ Real State will make with the proposed solution?

# IMPORTS

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# NOTEBOOK CONFIGS

In [2]:
pd.set_option('display.max_columns', None)

# HELPER FUNCTIONS

# 1.0 - LOADING DATA

In [3]:
df_raw = pd.read_csv(r'C:\Users\AlvaroMatsuda\OneDrive - DHAUZ\Área de Trabalho\DHAUZ_Files\Vivareal_Webscraping\scraped_data\df_webscraped.csv').drop(columns='Unnamed: 0')
df_raw.head()

,endereco,area,quartos,banheiros,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte,scrap_datetime
0,"Rua Francisco Pedro do Amaral, 251 - Jaguaré, ...",68m²,3 quartos,2 banheiros 1 suíte,2 vagas,R$ 500.000,R$ 575,"Piscina, Churrasqueira, Elevador, Academia, Ac...",2548091356,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
1,"Rua Santo Antônio, 670 - Bela Vista, São Paulo...",44m²,1 quarto,1 banheiro,Não informado Solicitar,R$ 289.000,R$ 319,"Aceita animais, Elevador, Portão eletrônico",2552171953,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
2,"Rua Domingos Rodrigues, 509 - Lapa, São Paulo ...",54m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 550.000,R$ 800,"Piscina, Mobiliado, Churrasqueira, Elevador, A...",2544567013,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
3,"Rua Humberto I, 193 - Vila Mariana, São Paulo ...",172m²,4 quartos,5 banheiros 3 suítes,4 vagas,R$ 4.810.000,R$ 2.100,"Piscina, Churrasqueira, Academia, Aceita anima...",2543253547,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
4,"Rua Quariteré, 267 - Parque da Mooca, São Paul...",78m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 680.000,R$ 1.000,"Piscina, Churrasqueira, Elevador, Condomínio f...",2550615565,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947


# 2.0 - DATA CLEANING

In [30]:
df02 = df_raw.copy()

In [184]:
df02 = df_raw.copy()

# Drop Duplicates -> Droped 2369 rows
df02.drop_duplicates(inplace=True)

# Select only address with São Paulo - SP -> droped 1 row
df02 = df02[df02['endereco'].str.contains('São Paulo - SP')]

# =============== 1.0 - area ===============
# Remove "m²"
df02['area'] = df02['area'].str.strip('m²')

# Drop "sem metrage"
df02.drop(df02[df02['area'] == 'sem metrage'].index, inplace=True)

# Convert to int
df02['area'] = df02['area'].astype('int64')

# =============== 2.0 - quartos ===============
# remove "quartos"
df02['quartos'] = df02['quartos'].str.strip('quartos')

# Convert to int
df02['quartos'] = df02['quartos'].astype('int64')

# =============== 3.0 - suite ===============
# create suite column
df02.insert(4, 'suite', df02[df02['banheiros'].str.contains('suíte')]['banheiros'].str.replace(' ', '').str[9:12].str.strip('suo'))

# fill NA - suite
df02['suite'].fillna(0, inplace=True)

# Convert to int
df02['suite'] = df02['suite'].astype('int64')

# =============== 4.0 - banheiros ===============
# extract number of bathrooms
df02['banheiros'] = df02['banheiros'].str[0]

# set "Não informado" = 1
df02.loc[df02['banheiros'] == 'N','banheiros'] = '1' 

# Convert to int
df02['banheiros'] = df02['banheiros'].astype('int64')

# =============== 5.0 - vaga_garagem ===============
# extract number of parking lots
df02['vaga_garagem'] = df02['vaga_garagem'].str[0]

# set "Não informado" = 0
df02.loc[df02['vaga_garagem'] == 'N','vaga_garagem'] = '0'

# Convert to int
df02['vaga_garagem'] = df02['vaga_garagem'].astype('int64')

# =============== 6.0 - valor_aluguel ===============
# remove "R$" and "."
df02['valor_aluguel'] = df02['valor_aluguel'].apply(lambda x: x.replace('R$', '').replace('.', ''))

# drop rows that have rent price instead of sell price
df02.drop(df02[df02['valor_aluguel'].str.contains('/')].index, inplace=True)

# Convert to int
df02['valor_aluguel'] = df02['valor_aluguel'].astype('int64')

# =============== 7.0 - valor_cond ===============
# remove "R$" and "."
df02['valor_cond'] = df02['valor_cond'].apply(lambda x: x.replace('R$', '').replace('.', ''))

# set "sem informacao" = 0
df02.loc[df02['valor_cond'] == 'sem informacao','valor_cond'] = '0'

# Convert to int
df02['valor_cond'] = df02['valor_cond'].astype('int64')

# =============== 8.0 - caracteristicas ===============



# 3.0 - DATA DESCRIPTION

## 3.1 - Rename Columns

## 3.2 - Data Dimentions

## 3.3 - Data Types

## 3.4 - Check NAs

## 3.5 - Descriptive Statistics

### 3.5.1 - Numerical Variables

### 3.5.2 - Categorical Variables

# 4.0 - CHECK OUTLIERS

# 5.0 - Hypothesis

## 5.1 - Mind Map Hypothesis

## 5.2 - Hypothesis Creation

# 6.0 - FEATURE ENGINEERING

# 7.0 - EDA

## 7.1 - Univariate Analysis

### 7.1.1 - Target Variable

### 7.1.2 - Numerical Variables

### 7.1.3 - Categorial Variables

## 7.2 - Bivariate Analysis

In [ ]:
# Validate Hypothesis

## 7.3 - Multivariate Analysis

### 7.3.1 - Numerical Variables

### 7.3.2 - Categorical Variables

# 8.0 - DATA PREPARATION

## 8.1 - Data Filtering

### 8.1.1 - Remove Outliers

### 8.1.2 - Drop Unnecessary Rows and Columns

## 8.2 - Train and Test Split

## 8.3 - Rescale/Encode Variables

### 8.3.1 - Train Dataset

### 8.3.2 - Test Dataset

# 9.0 - FEATURE SELECTION

# 10.0 - MACHINE LEARNING MODELLING

## 10.1 - Baseline Model

In [ ]:
# Usar a média do valor/m² por região/bairro como Baseline

## 10.2 - Modelos (Colocar o nome dos modelos)

## 10.3 - Models Performance - Comparison

# 11.0 - CROSS-VALIDATION SCORING

# 12.0 - HYPERPARAMETER FINE TUNNING

## 12.1 - Final Model with Best Parameters

# 13.0 - TRANSLATION AND INTERPRETATION OF THE ERROR

## 13.1 - Business Performance

## 13.2 - Machine Learning Performance on Test Dataset

# 14.0 - DEPLOY MODEL TO PRODUCTION

## 14.1 - Vivareal Class

## 14.2 - API Handler

## 14.3 - API Tester